In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import string
import nltk
import string
import numpy as np
import json
import pandas as pd
import math
import re
import contractions
from autocorrect import Speller

from nltk.corpus import reuters, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from slangs import slangs_dict

from transformers import pipeline

fix_spelling = pipeline("text2text-generation",model="grantslewis/spelling-correction-english-base-location-unique-2-2-proportional")

def get_user_tweets(user):

    url = f'https://twitter.com/{user}'

    # Define the options for the Chrome webdriver
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--incognito")
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument("--enable-javascript")

    driver = webdriver.Chrome(options=options)

    # Load the page
    driver.get(url)

    # Wait for tweets to populate the page
    try:
        WebDriverWait(driver, 5).until(expected_conditions.presence_of_element_located(
            (By.CSS_SELECTOR, '[data-testid="tweet"]')))
    except WebDriverException:
        return

    # Extract the HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    posts = soup.find_all(attrs={"data-testid": "tweetText"})

    # Extract the text content of each post
    post_content = [post.text for post in posts]

    # Save the data in a CSV file
    data = pd.DataFrame(post_content)

    driver.quit()

    return(data)

def get_tweets():
    df = pd.read_csv('./data/users.csv', delimiter=',')

    tweets = pd.DataFrame()

    for i in range(0, len(df)):
        if i % 10 == 0:
            print(i,'/',len(df)) 

        tweets = get_user_tweets(df['Twitter_username'][i])

        try:
            tweets.to_csv('./data/twitter_posts.csv', mode='a',index=False, header=False)
        except:
            continue

def clean_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df.apply(preprocess, axis=1)
    # df.apply(clean_text, axis=1)
    return df

def preprocess(text_to_clean):
    try:
        step = 'slang'        
        text_to_clean['text'] = ' '.join([slangs_dict[word.lower()] if word.lower() in slangs_dict.keys() else word for word in text_to_clean['text'].split()])

        step = 'contractions'
        text_to_clean['text'] = ' '.join([contractions.fix(word) for word in text_to_clean['text'].split()])

        step = 'attached'
        text_to_clean['text'] = re.sub(r'(?<=[^\s])(?=[A-Z])', ' ', text_to_clean['text'])

        step = 'numbers and words'
        text_to_clean['text'] = re.sub(r'(?<=\D)(?=\d)|(?<=\d)(?=\D)', ' ', text_to_clean['text'])

        step = 'regex'
        text_to_clean['text'] = re.sub('@[^\s]+',' ',text_to_clean['text'])
        text_to_clean['text'] = re.sub('http[^\s]+',' ',text_to_clean['text'])
        text_to_clean['text'] = re.sub('[^A-Za-z0-9\' ]+',' ',text_to_clean['text'])

        step = 'spaces'
        text_to_clean['text'] = re.sub(r'\s+', ' ', text_to_clean['text']).strip()

        step = 'spell'
        text_to_clean['text'] =  fix_spelling(text_to_clean['text'],max_length=2048)[0]['generated_text']

        step = 'contractions2'
        text_to_clean['text'] = ' '.join([contractions.fix(word) for word in text_to_clean['text'].split()])
        text_to_clean['text'] = re.sub('[^A-Za-z\' ]+',' ',text_to_clean['text'])
        
        step = 'spaces'
        text_to_clean['text'] = re.sub(r'\s+', ' ', text_to_clean['text']).strip()

        return text_to_clean

    except Exception as e:
        print(f'Failed at {step} with: {text_to_clean}, {e}')


def clean_text(text_to_clean):
    try:
        punctuation_list = list(string.punctuation)

        step = 'token'
        text_to_clean['text'] = word_tokenize(text_to_clean['text'])

        step = 'stop'
        text_to_clean['text'] = [word for word in text_to_clean['text'] if word not in stopwords.words('english')]

        step = 'punct'
        text_to_clean['text'] = [word for word in text_to_clean['text'] if word not in punctuation_list]

        step = 'lower'
        text_to_clean['text'] = [word.lower() for word in text_to_clean['text']]
       
        return  text_to_clean['text']
    except Exception as e:
        print(f'Failed at {step} with: {text_to_clean}, {e}')

/home/billy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tweets = pd.read_csv('./data/hugginoutput.csv')
tweets

,text,setiment
0,"Ngoc Pham, a 83 yr old Vietnamese man, was one...",negative
1,I’m making the following announcement and form...,positive
2,“The most aggressive field operation in Califo...,positive
3,Thank you Supervisor @AaronPeskin for endorsin...,positive
4,#tbt sharing a meal w/my friend @narendramodi ...,positive
...,...,...
1304,He will be a President for all of our families.,positive
1305,One week.,neutral
1306,How it started: How it's going:,neutral
1307,Joe’s ready,neutral


In [3]:
tweets = clean_df(tweets)
tweets

,text,setiment
0,Ngoc Pham A year old Vietnamese man was one of...,negative
1,I am making the following announcement and for...,positive
2,The most aggressive field operation in Califor...,positive
3,Thank you supervisor Aaron Peskin for endorsin...,positive
4,Tbt sharing a meal with my friend Wishing him ...,positive
...,...,...
1304,He will be a President for all of our families,positive
1305,One week,neutral
1306,How it started How it Is,neutral
1307,Joe S ready,neutral


In [4]:
tweets.to_csv('./data/clean.csv')